# DeutscheWelle Data Engineering Challenge

9.05.2023


# Task 1

## Connect via Gmail API



In [ ]:
## Script that dowloads emails from gmail inbox

## from https://developers.google.com/gmail/api/reference/rest/v1/users.messages.attachments/get 
# # Method to download email attachments from gmail api
##
# userId dwdechallenge@gmail.com / me (authenticated user)
# messageId 
# id  2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv 
#GET https://gmail.googleapis.com/gmail/v1/users/{userId}/messages/{messageId}/attachments/{id}

In [8]:
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials
from googleapiclient.errors import HttpError
import base64
import os
import time

In [9]:
#works! 
## 1) find email for the certain query i.e. 'report*csv' in attachment name
## 2) get message ids
## 3) get attachments ids
## 4) downloads attachments for every found email

def search_email(service, query_string,label_ids=[]):
    #try:
        message_list_response = service.users().messages().list(
            userId='me',
            labelIds=label_ids,
            q=query_string
        ).execute()

        message_items=message_list_response.get('messages')
        nextPageToken=message_list_response.get('nextPageToken')

        while nextPageToken:
            message_list_response = service.users().messages().list(
                    userId='me',
                    labelIds=label_ids,
                    q=query_string,
                    pageToken=nextPageToken
            ).execute()    

            message_items.extend(message_list_response.get('messages'))
            nextPageToken=message_items.get('nextPageToken')
        return message_items
    #except Exception as e:
    #    return 'No emails found'

def get_message_detail(msg_id,msg_format='metadata',metadata_headers: list=None):
    message_detail=service.users().messages().get(
        userId='me',
        id=msg_id,
        format=msg_format,
        metadataHeaders=metadata_headers
    ).execute()
    return message_detail

def get_file_data(message_id,attachment_id,file_name,save_location):
    response=service.users().messages().attachments().get(
        userId='me',
        messageId=message_id,
        id=attachment_id
    ).execute()

    file_data=base64.urlsafe_b64decode(response.get('data').encode('UTF-8'))
    return file_data
    


# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']
creds = Credentials.from_authorized_user_file('../token.json', SCOPES)
service = build('gmail', 'v1', credentials=creds)

## search for emails with 'report' in the attachment name
#query_string='has:attachment' # all email with attachments
query_string='filename:report*csv' #find all the email with attachment in csv format containing 'report' in their name
#query_string='report' # all email with report in subject/body text
email_messages = search_email(service, query_string,['INBOX'])
#print(email_messages)
# try:
#     path = os.path.join('./', 'Downloaded_files')
#     os.mkdir(path)
# except OSError as error:
#     print(error) 
save_location='../data/downloaded'
## download attachments for found emails 

for email_message in email_messages:
    #print(email_message['id'])
    #187ffb2fbf3c1bc5
    messageDetail = get_message_detail(email_message['id'],msg_format='full',metadata_headers=['parts'])
    #print(messageDetail)
    messageDetailPayload = messageDetail.get('payload')

    if 'parts' in messageDetailPayload:
        for msgPayload in messageDetailPayload['parts']:
            file_name = msgPayload['filename']
            body=msgPayload['body']

            if 'attachmentId' in body:
                attachment_id = body['attachmentId']
                attachment_content= get_file_data(email_message['id'],attachment_id,file_name,save_location)
                print(file_name)
                
                #with open(os.path.join(save_location + '/' 'Downloaded_files' + '/' + file_name),'wb') as _f:
                with open(os.path.join(save_location + '/' + file_name),'wb') as _f:
                    _f.write(attachment_content)
                    print(f'File {file_name} is saved at {save_location}')
    time.sleep(0.5)
    #break

[Errno 17] File exists: './Downloaded_files'
2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv
File 2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv is saved at ./Downloaded_files
2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv
File 2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv is saved at ./Downloaded_files
2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv
File 2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv is saved at ./Downloaded_files
2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv
File 2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv is saved at ./Downloaded_files
2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv
File 2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2

## Read the data from the files and upload them to the postgres database.

In [19]:
import pandas as pd
import psycopg2
import os
import glob



In [20]:
#Write a python script that identifies the correct daily reports

#!ls /Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/

#daily_repo_filepath= "/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/daily_reports/"
daily_repo_filepath='./Downloaded_files' # downloaded files via API
#file1='2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv'
files=glob.glob(os.path.join(daily_repo_filepath,"*.csv"))
##date


In [21]:
files

['./Downloaded_files/2022-09-18-23-03-32-EDT-Historical-Report-ENTRFacebook-2022-06-19--2022-09-19.csv',
 './Downloaded_files/2022-09-16-23-04-01-EDT-Historical-Report-ENTRFacebook-2022-06-17--2022-09-17.csv',
 './Downloaded_files/2022-09-21-23-04-26-EDT-Historical-Report-ENTRFacebook-2022-06-22--2022-09-22.csv',
 './Downloaded_files/2022-09-17-23-04-42-EDT-Historical-Report-ENTRFacebook-2022-06-18--2022-09-18.csv',
 './Downloaded_files/2022-09-20-23-03-35-EDT-Historical-Report-ENTRFacebook-2022-06-21--2022-09-21.csv',
 './Downloaded_files/2022-09-15-23-04-08-EDT-Historical-Report-ENTRFacebook-2022-06-16--2022-09-16.csv',
 './Downloaded_files/2022-09-23-23-04-15-EDT-Historical-Report-ENTRFacebook-2022-06-24--2022-09-24.csv']

In [27]:
data= pd.read_csv(files[0])
#data= [pd.read_csv(file) for file in files]
# data=pd.read_csv(daily_repo_filepath+files)

In [28]:
data

,Page Name,User Name,Facebook Id,Page Category,Page Admin Top Country,Page Description,Page Created,Likes at Posting,Followers at Posting,Post Created,...,Message,Link,Final Link,Image Text,Link Text,Description,Sponsor Id,Sponsor Name,Sponsor Category,Overperforming Score
0,Onet Styl Życia,onetstylzycia,372341662829366,NEWS_SITE,PL,Najbardziej stylowy serwis w Polsce!,2012-06-05 16:57:21,558890,563792,2022-09-18 21:00:26 EDT,...,Wyrwał: w rękach Rosjan jest ok. 17 proc. tery...,https://www.facebook.com/onetstylzycia/videos/...,NaN,NaN,"""To jest przełom w wojnie w Ukrainie, zdobywam...",NaN,NaN,NaN,NaN,-0.89
1,DW News,deutschewellenews,100064359716848,NEWS_SITE,DE,Follow us for trustworthy news and analysis fr...,2008-08-26 12:26:42,2664421,3150873,2022-09-18 20:12:16 EDT,...,Every year 400 million tonnes of plastic are p...,https://www.facebook.com/deutschewellenews/vid...,NaN,NaN,Highlighting the plastic waste problem,NaN,NaN,NaN,NaN,-3.29
2,DW Euromaxx,dw.euromaxx,211745678846355,BROADCASTING_MEDIA_PRODUCTION,DE,Euromaxx is made for those who love European #...,2011-05-17 14:08:01,501640,756194,2022-09-18 19:15:16 EDT,...,"🥚🥔 In Spain, they're eaten for breakfast, lunc...",https://www.facebook.com/dw.euromaxx/videos/47...,NaN,NaN,The Secrets Behind the Traditional Spanish Tor...,NaN,NaN,NaN,NaN,1.01
3,Onet Wiadomości,OnetWiadomosci,56024223012,NEWS_SITE,PL,"Jeśli macie jakieś uwagi, chcecie pokazać, jak...",2009-01-13 10:54:16,533859,625613,2022-09-18 19:00:20 EDT,...,"""Staram się, aby to nie była zwykła relacja z ...",https://www.facebook.com/OnetWiadomosci/videos...,NaN,NaN,"""Robię filmiki komediowe, podróżnicze, ciekawo...",NaN,NaN,NaN,NaN,-5.17
4,Konbini news,konbininews,183081005507412,MEDIA_NEWS_COMPANY,FR,Society & Politics by Konbini,2017-01-30 10:03:14,722701,1546605,2022-09-18 16:05:01 EDT,...,"Algorithme, limites entre vie pro et perso, me...",https://buff.ly/3BHd46t,https://www.konbini.com/videos/enjoyphoenix-de...,NaN,EnjoyPhoenix dévoile les réalités du métier de...,"Algorithme, limites entre vie pro et perso, me...",NaN,NaN,NaN,-4.72
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7066,ENTR ro,ro.entr,100068014577172,BROADCASTING_MEDIA_PRODUCTION,DE,"Tânăr, curajos, european. ENTR este un spațiu ...",2021-04-12 09:27:16,3160,3804,2022-06-19 02:10:05 EDT,...,„Dacă o să ne uităm la cantitatea de deșeuri g...,https://www.facebook.com/ro.entr/videos/439191...,NaN,NaN,De ce România reciclează doar 15% din deșeuri,NaN,NaN,NaN,NaN,-10.50
7067,Noizz Food,NoizzFood,975020859234684,WEBSITE,PL,Zainspiruj się naszymi przepisami i poradami n...,2016-02-09 12:37:25,1092438,1286335,2022-06-19 02:03:00 EDT,...,"Śniadanie zjedzone, można myśleć o obiedzie - ...",https://www.facebook.com/NoizzFood/videos/5833...,NaN,NaN,Buraczkowe penne z parmeznem,NaN,NaN,NaN,NaN,-1.77
7068,RFI Romania,RFI.Romania.FM,187935071639,NEWS_SITE,FR,Pagina oficială RFI România. Puteţi să ne urmă...,2009-11-26 07:57:41,85515,96074,2022-06-19 02:01:06 EDT,...,"Raluca Avram, o tânără din Cluj-Napoca, pasion...",https://www.facebook.com/RFI.Romania.FM/videos...,NaN,NaN,"Interviu cu Raluca, o fată specială, despre mo...",NaN,NaN,NaN,NaN,-11.60
7069,ENTR en,en.entr,102500518635499,BROADCASTING_MEDIA_PRODUCTION,FR,"Young, bold, European. ENTR a digital space fo...",2021-04-12 09:22:28,469,889,2022-06-19 00:45:00 EDT,...,The EU wants to ban the sell of combustion car...,https://www.facebook.com/en.entr/videos/574913...,NaN,NaN,The end of fossil fuelled cars?,NaN,NaN,NaN,NaN,-27.00


In [29]:
# connect to postgres database
param_dic = {
    "host"      : "localhost",
    "database"  : "postgres",
    "user"      : "postgres",
}
def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1) 
    print("Connection successful")
    return conn

In [30]:
conn = connect(param_dic)

def execute_query(conn, query):
    """ Execute a single query """
    connect(param_dic)
    ret = 0 # Return value
    cursor = conn.cursor()
    try:
        cursor.execute(query)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1

    # If this was a select query, return the result
    if 'select' in query.lower():
        ret = cursor.fetchall()
    cursor.close()
    return ret

Connecting to the PostgreSQL database...
Connection successful


In [31]:
#create postgres database

cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])
new_table='new_table'

#query=("DROP TABLE IF EXISTS %s;"%(new_table))
query=("CREATE TABLE IF NOT EXISTS %s(%s)" %(new_table,(cols))) 

execute_query(conn, query)

Connecting to the PostgreSQL database...
Connection successful


0

In [ ]:
# 2nd option create postgres database
def create_postgres (db,new_table):
    cols=cols=','.join([column.replace(" ", "_").replace("?","").lower() + str(' VARCHAR') for column in data.columns.values])

    conn = connect(param_dic)
    cur = conn.cursor()

    #drop table if already exists
    cur.execute("DROP TABLE IF EXISTS %s;"%(new_table))
    #create database
    cur.execute("CREATE TABLE IF NOT EXISTS %s(%s)" %(new_table,(cols))) 
    conn.commit()
    conn.close()

In [32]:
## Upload data to the postgres database

def upload_csv_to_postgres(table, csv):
    conn = connect(param_dic)
    cur = conn.cursor()
    try:
        [cur.copy_expert("COPY %s FROM STDIN WITH DELIMITER ',' CSV "%(table), open(csv_file)) for csv_file in files]
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cur.close()
        return 1

    conn.commit()
    conn.close()

In [34]:
table='new_table'

#create_postgres(data,table)
#upload_csv_to_postgres(table,files)
[upload_csv_to_postgres(table,files) for file in files]


Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful
Connecting to the PostgreSQL database...
Connection successful


[None, None, None, None, None, None, None]

In [ ]:
conn = connect(param_dic)
execute_query(conn, 'SELECT Page_Name User_Name FROM new_table')

In [ ]:
##SQL script to process the data and push the data to GIT
## create tables 


# Task 2

#### Have a look at the dwh_dl_facebook_post_insights.csv. You will find there 3 rows for seperate posts. Import the csv to a database table

In [ ]:
path_insights='/Users/anya/SCIENCE/DW_Challenge/Junior_DE_Task/'
db_insights=pd.read_csv(path_insights+'dwh_dl_facebook_post_insights.csv')

In [ ]:
db_insights

In [ ]:
files

In [ ]:

table='insights_draft'
data=db_insights
files=glob.glob(os.path.join(path_insights,"*.csv"))
#conn = connect(param_dic)
create_postgres(data,table)
upload_csv_to_postgres(table,files)

In [ ]:
files

### Please write a SQL script to unpack the json in column post_video_view_time_by_age_bucket_and_gender in a new table, while preserving the other columns.

In [ ]:
SELECT 
   id, 
   data::json->'name' as name,
   data::json->'author' ->> 'last_name' as author
FROM books;

In [ ]:
conn = connect(param_dic)

sel_column='post_video_view_time_by_age_bucket_and_gender'
table='insights_draft'
execute_query(conn,'SELECT %s FROM %s'%(sel_column,table))

In [ ]:
execute_query(conn,'SELECT %s FROM %s'%(sel_column,table))

In [ ]:
execute_query(conn, "SELECT %s  FROM %s, data::jsons->'U.18-24' as U.18-24, data::jsons->'U.25-34' as U.25-34"%(sel_column,table))

In [ ]:
execute_query(conn,'SELECT %s FROM %s, json_each(%s)'%(sel_column,table,sel_column))

In [ ]:
print(sel_column)
conn = connect(param_dic)
execute_query(conn, "SELECT %s json_data -> '%s' %s, FROM %s;" %(sel_column,sel_column,'foo',table))

In [ ]:
table

In [ ]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    SELECT json_data 
        post_video_view_time_by_age_bucket_and_gender -> "U.18-24" as U1
    FROM insights_draft;
    """)
conn.commit()
conn.close()

In [ ]:
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres")
cur = conn.cursor()
cur.execute("""
    SELECT (post_video_view_time_by_age_bucket_and_gender) from insights_draft;
    """)
conn.commit()
conn.close()

In [ ]:
execute_query(conn,'select jsonb_object_keys(%s) from %s;'%(sel_column,table))

In [ ]:
table

In [ ]:
print(sel_column)
conn = connect(param_dic)
execute_query(conn, "SELECT %s -> 'U.18-24' AS U1 FROM %s" %(sel_column,table))

#SELECT post_video_view_time_by_age_bucket_and_gender FROM insights_draft;

#SELECT post_video_view_time_by_age_bucket_and_gender json_data ->> 'U.18-24' AS U1 FROM insights_draft; 

In [ ]:
execute_query(conn,"select 
    json_data #> 
       '{pizzas,1,additionalToppings}'  
        as additional_toppings_2nd_pizza
from test;")

### Write a script for the sums per post for each gender. Additionally create the sum for age 18-34 of all genders.

In [ ]:
# SELECT SUM(Quantity) FROM OrderDetails;
# add where statement WHERE Country='Mexico' WHERE Country='Mexico'
# SELECT * FROM Products WHERE Price BETWEEN 50 AND 60;